In [136]:
def calc_span(paragraph, sentence):
    if isinstance(sentence, dict):
        if sentence.__contains__('text'):
            text = sentence.get('text') 
            if text in paragraph:
                start = paragraph.find(text)
                end = paragraph.find(text) + len(text)
                return [start, end]
            else:
                return [0,0]
        else:
            return [0,0]
    elif isinstance(sentence, str):
        if sentence in paragraph:
            start = paragraph.find(text)
            end = paragraph.find(text) + len(text)
            return [start, end]
        else:
            return [0,0]
    else:
        return [0,0]

                 

In [91]:
def convert_to_eval_format(paragraph, prediction):
    span = calc_span(paragraph, prediction)
    if isinstance(span, list):
        return {paragraph: {prediction['class']: span}}
    else:
        return "invalid"

In [146]:
def convert_list_to_eval_format(paragraph, predictions):
    span_list = []
    for prediction in predictions:
        span = calc_span(paragraph, prediction)
        span_list.append({prediction['class']: span})
    return {paragraph: span_list}

In [8]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
import json

with open('../Data/gold_standard_dataset.jsonl', 'r') as file:
    data = [json.loads(line) for line in file]

In [57]:
text = data[1]['text']
# example prediction from LLM
pred = {"text": "That leads to me believe that most cat lovers are really shy.", "class": "hasty generalization"}
print(calc_span(text, pred))
print(convert_to_eval_format(text, pred))

[84, 145]
{'Two of my best friends are really introverted, shy people, and they both have cats. That leads to me believe that most cat lovers are really shy.\n': {'hasty generalization': [84, 145]}}


In [148]:
#convert csv results to eval json
import csv

# Read CSV file
file_path = 'df_results_gemma_multiclass (3).csv'

file = open(file_path, mode = 'r', newline = '', encoding='utf8')
csv_reader = csv.reader(file, delimiter=',')

#skip header
next(csv_reader)
# Read the rest of the rows
data = [row for row in csv_reader]

### Multi-label

In [152]:
import json

input = [row[1] for row in data]
predictions = [json.loads(row[3]) for row in data]
eval_data = []
for n, paragraph in enumerate(input):
    eval_data.append(convert_list_to_eval_format(paragraph, predictions[n]))

### Multi-class

In [171]:
input = [row[1] for row in data]
predictions = [json.loads(row[3]) for row in data]
eval_data = []
for n, paragraph in enumerate(input):
    prediction = predictions[n]
    if prediction:
        eval_data.append({paragraph: prediction[0]})
    else:
        eval_data.append({paragraph: 0})


### Write to file

In [175]:
file_name = file_path.replace('.csv', '.json')
with open(f'../Data/formatted_results/eval_{file_name}', 'w') as out:
    json.dump(eval_data , out)